In [ ]:
#@title Installing packages
!pip install optuna

In [24]:
#@title Preparing the data
# Loading the libraries
import numpy as np
import pandas as pd
pd.set_option("display.precision", 4)
import tensorflow as tf
import tensorflow.keras.layers as F
import optuna

# Classification models
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

# Importing the data
df = pd.read_csv('Data.csv')
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

## Encoding the categorical data
# Label encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

# One-Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])],
                       remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

# Split the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,
                                                    random_state = 42)

# Feature Scaling (a must for NN)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
#@title Training the Classification models on Bag-of-words 

def list_classifiers():
    classifiers = {
        'Logistic Regression' : LogisticRegression(random_state = 42),
        'K-Nearest Neighbors' : KNeighborsClassifier(n_neighbors = 5, p = 2, 
                                                     metric = 'minkowski'),
        'Support Vector Machine' : SVC(kernel = 'linear', random_state = 42),
        'Kernel SVM' : SVC(kernel = 'rbf', random_state = 42),
        'Naive Bayes' : GaussianNB(),
        'Decision Tree' : DecisionTreeClassifier(criterion = 'entropy', 
                                                 random_state = 42),
        'Random Forest' : RandomForestClassifier(n_estimators=100, 
                                                 criterion='entropy',
                                                 random_state = 42),
    }
    return classifiers

# Get the classifiers
classifiers = list_classifiers()
score = {name : [] for name in list(classifiers)} 
for name, classifier in classifiers.items():
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  score[name].append(f1_score(y_test, y_pred))
  score[name].append(accuracy_score(y_test, y_pred))

In [9]:
#@title NN without HP tuning
# Building the model
model = tf.keras.models.Sequential()
model.add(F.Dense(units = 6, activation = 'relu')) # Input and first hidden layer
model.add(F.Dense(units = 6, activation = 'relu')) # Second hidden layer
model.add(F.Dense(units = 1, activation = 'sigmoid')) # Output layer

# Training the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 32, epochs = 100, verbose = 0)

# Predicting the test values
y_pred = np.rint(model.predict(X_test))

# f1 score and accuracy of predictions
score['NN normal'] = [f1_score(y_test, y_pred), 
                 accuracy_score(y_test, y_pred)]

In [10]:
#@title Simple Optuna NN Tuner

def objective(trial):
  # Build the model
  model = tf.keras.models.Sequential()
  n_layers = trial.suggest_int("n_layers", 1, 4)
  for i in range(n_layers):
      num_hidden = trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True)
      model.add(
          F.Dense(
              units = num_hidden,
              activation = 'relu'
          )
      )
  # Output layer
  model.add(F.Dense(units = 1, activation = 'sigmoid'))
  # Optimize the learning_rate for Adam optimizer    
  lr = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                loss = 'binary_crossentropy', metrics = ['accuracy'])
  
  # Training the model
  model.fit(X_train, y_train, batch_size = 32, epochs = 100, verbose = 0)
  # Predicting the test values
  y_pred = np.rint(model.predict(X_test))
  # Calculating the scores
  accuracy = accuracy_score(y_test, y_pred)
  return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize", 
                                study_name = 'Simple Optuna NN Tuning')
    study.optimize(objective, n_trials=100)

[I 2022-04-06 09:34:01,597] A new study created in memory with name: Simple Optuna NN Tuning
[I 2022-04-06 09:35:24,481] Trial 0 finished with value: 0.8552 and parameters: {'n_layers': 4, 'n_units_l0': 41, 'n_units_l1': 79, 'n_units_l2': 9, 'n_units_l3': 81, 'learning_rate': 1.5183727065372833e-05}. Best is trial 0 with value: 0.8552.
[I 2022-04-06 09:36:01,573] Trial 1 finished with value: 0.852 and parameters: {'n_layers': 1, 'n_units_l0': 65, 'learning_rate': 0.00843583609962644}. Best is trial 0 with value: 0.8552.
[I 2022-04-06 09:36:43,196] Trial 2 finished with value: 0.8612 and parameters: {'n_layers': 1, 'n_units_l0': 11, 'learning_rate': 0.0010410458850084682}. Best is trial 2 with value: 0.8612.
[I 2022-04-06 09:37:24,849] Trial 3 finished with value: 0.8596 and parameters: {'n_layers': 1, 'n_units_l0': 79, 'learning_rate': 0.0014316121947019827}. Best is trial 2 with value: 0.8612.
[I 2022-04-06 09:38:00,672] Trial 4 finished with value: 0.8224 and parameters: {'n_layers':

In [13]:
#@title NN with Optuna Tuner
# Take the results
lr, n_layers, n_units = list(study.best_trial.params.values())
# Building the model
model = tf.keras.models.Sequential()
for layer in range(n_layers):
  model.add(F.Dense(units = n_units, activation = 'relu'))

model.add(F.Dense(units = 1, activation = 'sigmoid')) # Output layer

# Training the model
model.compile(optimizer = tf.optimizers.Adam(learning_rate = lr), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 32, epochs = 100, verbose = 0)

# Predicting the test values
y_pred = np.rint(model.predict(X_test))

# f1 score and accuracy of predictions
score['NN tuned'] = [f1_score(y_test, y_pred), 
                 accuracy_score(y_test, y_pred)]

In [25]:
#@title Results
result = pd.DataFrame(score.values(), columns = ['f1 score', 'Accuracy'], 
                      index = score.keys())
result.style

,f1 score,Accuracy
Logistic Regression,0.2891,0.8072
K-Nearest Neighbors,0.4619,0.8304
Support Vector Machine,0.0000,0.8012
Kernel SVM,0.5182,0.8572
Naive Bayes,0.4709,0.8256
Decision Tree,0.5134,0.7968
Random Forest,0.5829,0.8672
NN normal,0.5711,0.8624
NN tuned,0.0000,0.8012
